# Simple PID Example
This example uses the Vicon the get the pose of the robot in the lab and uses a simple linear controller to command the position of the robot. The output of the controller is desired linear and angular velocities represented in the body coordinate frame and is fed to the B1's default controller. Following shows the control architecture:

## Instantiate a High-Level Interface to The Robot  

In [1]:
from B1Py.interfaces import B1HighLevelReal
import time
robot = B1HighLevelReal()

UDP Initialized. socketfd: 65   Port: 8080


If the connection was a success, running the following would lead to the robot jogging in its place:

In [2]:
for i in range(100):
    time.sleep(0.01)
    robot.setCommand(0.0,0,0)

### Let's Play a Bit, Controlling with Joystick
Now that we're connected to the robot, let's do a simple test and control ot using a Logitech 3D pro joystick. To communicate with the joystick, simply use the Pygame-based interface class provided in B1Py:

In [2]:
from B1Py.joysticks import Logitech3DPro
joy = Logitech3DPro(joy_id=0)

pygame 2.5.1 (SDL 2.28.2, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html
Calibrating joystick, please do not touch the stick and wait...
Put your stick at zero location and do not touch it
Calibration finished.


If the connection to the joystik was a success, running the following code should show the values read from the joystick:

In [ ]:
for i in range(500):
    time.sleep(0.01)
    print(joy.readAnalog())
    

Now that we can successfully read the joystick, use it to control the robot. We use the z axis of the joystick the yaw rate and the auxilary knob to modify the body height:

In [ ]:
for i in range(500):
    cmd = joy.readAnalog()
    vx = cmd['x']
    vy = cmd['y']
    w = cmd['z'] / 2
    body_height = cmd['aux'] / 10
    robot.setCommand(vx,vy,w, bodyHeight=body_height)
    time.sleep(0.01)


## Establish Interface to Vicon
Now that we have seen that we can control the robot with joystck, let's take another step and try to control it based on its measured pose as reported by Vicon. But before that, we need to establish our connection to the Vicon system and also apply the extrinsic transformation between the body frame and marker coordinate system. 

First, launch the ROS vicon bridge in an idependent terminal:
```bash
roslaunch vicon_bridge vicon.launch
```

Then use the provided python tool with P1Py to create a class with a constantly running thread to read the pose of the robot marker frame in the vicon world frame:

In [3]:
from B1Py.utils import addROS2Path
# Add ROS to the ptyhon path. 
addROS2Path('/opt/ros/noetic') 
from B1Py.ros_bridges import initRosNode, ROSTFListener
# Initialize the ROS node for the current python process
initRosNode('b1py_node')
# Create a TF listener object that listens to the transform of robot vicon mareker with respect to the world world_T_marker
robot_body_tf = ROSTFListener('vicon/world', 'vicon/b1_body_whole_1/b1_body_whole_1')

In [4]:
robot_body_tf.T

array([[ 0.99305802, -0.11701313,  0.01198734,  0.05164038],
       [ 0.11732368,  0.99264891, -0.02972015,  0.69296006],
       [-0.00842157,  0.03092023,  0.99948638,  0.78387855],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

Now read the calibration parameters from the Vicon2GT output file and use it get the pose of the robot body in vicon world frame:

In [4]:
import numpy as np
from B1Py.calibration import parseVico2GtParams
body_T_marker = parseVico2GtParams('notebooks/sample_params/vicon2gt/unitree_imu.txt')

For sanity check, use the provided TF publisher tool in B1Py to visualize the robot body frame in vicon world frame:

In [6]:
from B1Py.ros_bridges import ROSTFPublisher
import time
tf_pub = ROSTFPublisher('vicon/world', 'vicon/dog')

for i in range(300):
    world_T_marker = robot_body_tf.T
    T = world_T_marker @ np.linalg.inv(body_T_marker)
    tf_pub.publish(T)
    time.sleep(0.01)

In [7]:
T

array([[ 0.00494705, -0.99990835,  0.01263343, -0.22522719],
       [ 0.99967454,  0.00462917, -0.02508674,  0.18978976],
       [ 0.02502594,  0.01275346,  0.99960574,  0.17573586],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [8]:
from B1Py.ros_bridges import ROSTFPublisher
import pypose as pp
import time
tf_pub = ROSTFPublisher('vicon/world', 'vicon/doggg')

In [1]:
import threading
import pypose as pp
class PositionController():
    """
    Class responsible for the position Control of the robot through the 
    high-level interface (robot takes velocity and yaw angular rate commands).

    Attributes:
        loop_freq (int): The frequency of the control loop.
        robot (object): An instance of `B1HighLevelReal` class
        pose_sensor (object): An instance of `ROSTFListener` or similar classes.
        sensor_T_body (numpy.ndarray): 4x4 Transformation representing the pose of sensor with respect to body frame.
        Kp_yaw (float): Proportional gain for yaw control.
        Ki_yaw (float): Integral gain for yaw control.
        Kp_pos (float): Proportional gain for position control.
        Ki_pos (float): Integral gain for position control.
        user_callback (func): User-defined callback function executed before sending the control command to the robot.
    """
    def __init__(self, 
                robot, 
                pose_sensor, 
                body_T_sensor, 
                loop_freq=100, 
                Kp_yaw = 0.8,
                Ki_yaw = 0.02,
                Kp_pos = 0.8,
                Ki_pos = 0.02,
                user_callback = None):
        """
        Initialize a new PositionController.

        Parameters:
            robot (object): An instance of the `B1HighLevelReal` class or similar interfaces.
            pose_sensor (object): An instance of the `ROSTFListener` class or similar sensors.
            body_T_sensor (numpy.ndarray): 4x4 Transformation representing the pose of sensor with respect to body frame.
            loop_freq (int, optional): Frequency of the control loop. Defaults to 100.
            Kp_yaw (float, optional): Proportional gain for yaw control. Defaults to 0.8.
            Ki_yaw (float, optional): Integral gain for yaw control. Defaults to 0.02.
            Kp_pos (float, optional): Proportional gain for position control. Defaults to 0.8.
            Ki_pos (float, optional): Integral gain for position control. Defaults to 0.02.
            user_callback (func, optional): User-defined callback function. Defaults to None.
        """
        self.loop_freq=loop_freq
        self.robot = robot
        self.pose_sensor = pose_sensor
        self.sensor_T_body = np.linalg.inv(body_T_sensor)
        self.running = True
        self.jogging = False
        self.Kp_yaw = Kp_yaw
        self.Ki_yaw = Ki_yaw
        self.Kp_pos = Kp_pos
        self.Ki_pos = Ki_pos
        self.body_height = 0
        self.step_height = 0
        self.user_callback = user_callback
        self.setCurrentAsHome()
        self.control_thread = threading.Thread(target = self.controlLoop)
        self.control_thread.start()
        
    
    def getAbsolutePose(self):
        """
        Get the absolute pose of the robot as reported by the sensor with respect to the 
        perception/Vicon world frame.

        Returns:
            numpy.ndarray: The absolute 4x4 pose matrix.
        """
        T = self.pose_sensor.T @ self.sensor_T_body
        return T

    def getRelativePose(self):
        """
        Get the relative pose of the robot with respect to the home set during
        the instantiation of the class or latest execution of the setCurrentAsHome method.

        Returns:
            numpy.ndarray: The relative 4x4 pose matrix.
        """
        return self.T0_inv@self.getAbsolutePose()
    
    def setCurrentAsHome(self):
        """
        Set the current position and orientation as the home (reference) point.
        """
        self.T0_inv = np.linalg.inv(self.getAbsolutePose())
        self.p_des = np.array([0.,0.])
        self.yaw_des = 0.0

    def getState(self):
        """
        Retrieve the current state of the robot.

        Returns:
            tuple: Current x, y, and yaw of the robot.
        """
        T = self.getRelativePose() 
        yaw = self.pose2Yaw(T)
        x, y = T[0,-1], T[1,-1]
        return x, y, yaw

    def pose2Yaw(self, T):
        """
        Convert a transformation matrix to a yaw angle.

        Parameters:
            T (numpy.ndarray): Transformation matrix.

        Returns:
            float: Yaw angle in radians.
        """
        return pp.euler(pp.mat2SE3(T))[2].item()

    def go2Relative(self, x, y, heading):
        """
        Command the robot to move to a relative pose with respect
        to the current pose of the robot.

        Parameters:
            x (float): Relative x-coordinate.
            y (float): Relative y-coordinate.
            heading (float): Relative heading in radians.
        """
        self.setCurrentAsHome()
        self.p_des = np.array([x, y])
        self.yaw_des = heading

    def go2Absolute(self, x, y, heading):
        """
        Command the robot to move to an absolute pose with respect to
        the home pose defined during the instantiation of the class or 
        latest call to the `setCurrentAsHome` method.

        Parameters:
            x (float): Absolute x-coordinate.
            y (float): Absolute y-coordinate.
            heading (float): Absolute heading in radians.
        """
        self.p_des = np.array([x, y])
        self.yaw_des = heading
    
    def perf_controller(self):
        """
        Perform the control computations to get the command to be sent to the robot.

        Returns:
            tuple: Control commands vx, vy, and omega in robot body frame.
        """
        x, y, yaw = self.getState()
        # yaw controller
        R = pp.euler2SO3([0,0,yaw]).matrix()
        e_yaw = self.yaw_des - yaw
        yaw_cmd = self.Kp_yaw * e_yaw
        # translational controller
        e_x = self.p_des[0] - x
        e_y = self.p_des[1] - y
        x_cmd = self.Kp_pos*e_x
        y_cmd = self.Kp_pos*e_y
        # Rotate the command vector into the body frame
        cmd_in_world = np.array([x_cmd, y_cmd, 0]).reshape(3,1)
        cmd_in_body  = (R.T@cmd_in_world)[0:2] 
        u_perf = cmd_in_body[0].item(), cmd_in_body[1].item(), yaw_cmd
        return u_perf

    def startJogging(self):
        """
        Activate the control, the robot starts jogging.
        """
        self.jogging = True
    
    def stopJogging(self):
        """
        Deactivate the control, the robot stops jogging.
        """
        self.jogging = False

    def close(self):
        """
        Stop the control loop and terminate the thread.
        """
        self.running = False
        self.control_thread.join()

    def setBodyHeightOffset(self, h):
        """
        Set the height offset for the robot's body 
        with respect to the nominal height.

        Parameters:
            h (float): Height offset.
        """
        self.body_height = h

    def setStepHeightOffset(self, h):
        """
        Set the swing foot height offset.

        Parameters:
            h (float): Step height offset.
        """
        self.step_height = h

    def controlLoop(self):
        """
        Control loop that continuously updates the robot's state and issues control commands.
        """
        while self.running:
            emergency_stop = self.robot.state.wirelessRemote[2]
            if emergency_stop:
                print("User pressed the E-Stop switch, terminating the controller ...")
                self.running = False
            if self.jogging:
                cmd = self.perf_controller()
                # In case we need a CBFQP filter
                if self.user_callback is not None:
                    cmd = self.user_callback(cmd)
                #clip the max speed to some safe limits
                cmd = np.clip(np.array(cmd), -1,1)
                self.robot.setCommand(cmd[0], cmd[1], cmd[2], bodyHeight = self.body_height, 
                                                              footRaiseHeight = self.step_height,  
                                                              mode = 2)
            else:
                self.robot.setCommand(0, 0, 0, bodyHeight = self.body_height, 
                                               footRaiseHeight = self.step_height,  
                                               mode = 0)
            time.sleep(1/self.loop_freq)

In [55]:
b1_controller = PositionController(robot, robot_body_tf, body_T_marker)

User pressed the E-Stop switch, terminating the controller ...


In [21]:
b1_controller.startJogging()

In [22]:
b1_controller.stopJogging()

In [8]:
b1_controller.go2Relative(-0.0,0.5, -0)

In [18]:
b1_controller.go2Absolute(0.0,0.0, 0.8)

In [17]:
b1_controller.setBodyHeightOffset(-0.05)

In [27]:
b1_controller.close()